In [ ]:
!mkdir Fashion_MNIST
!curl -L -o /content/Fashion_MNIST/fashionmnist.zip https://www.kaggle.com/api/v1/datasets/download/zalando-research/fashionmnist
!unzip /content/Fashion_MNIST/fashionmnist.zip -d /content/Fashion_MNIST/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 68.7M  100 68.7M    0     0  14.3M      0  0:00:04  0:00:04 --:--:-- 16.5M
Archive:  /content/Fashion_MNIST/fashionmnist.zip
  inflating: /content/Fashion_MNIST/fashion-mnist_test.csv  
  inflating: /content/Fashion_MNIST/fashion-mnist_train.csv  
  inflating: /content/Fashion_MNIST/t10k-images-idx3-ubyte  
  inflating: /content/Fashion_MNIST/t10k-labels-idx1-ubyte  
  inflating: /content/Fashion_MNIST/train-images-idx3-ubyte  
  inflating: /content/Fashion_MNIST/train-labels-idx1-ubyte  


In [ ]:
!pip install tensorinfo


ERROR: Could not find a version that satisfies the requirement tensorinfo (from versions: none)
ERROR: No matching distribution found for tensorinfo


In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/Fashion_MNIST/fashion-mnist_train.csv')
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25, random_state=69)
X_train = X_train/255.0
x_test = X_test/255.0

from torch.utils.data import Dataset, DataLoader
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features , dtype = torch.float32).reshape(-1,1,28,28)
    self.labels = torch.tensor(labels, dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self, index):
    return self.features[index], self.labels[index]

train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory= True)
test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory= True)

In [28]:
import torch.nn as nn
class NN(nn.Module):
  def __init__(self, features):
    super().__init__()

    self.features =   nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3, padding = 'same'),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(32,64,kernel_size=3, padding = 'same'),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.classifiers = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7,128), # 1*28*28 -> 32*28*28 -> 32*14*14 -> 64*14*14 -> 64*7*7
        nn.ReLU(),
        nn.Dropout(0.35),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Dropout(0.35),
        nn.Linear(64,10)
    )


  def forward(self,x):
    x = self.features(x)
    x = self.classifiers(x)
    return x

In [ ]:
learning_rate = 0.01
epochs = 100

In [30]:
mymodel = NN(1)
mymodel.to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mymodel.parameters(), lr=learning_rate)

In [31]:
for epoch in range(epochs):
  total_epochs_loss = 0
  for data, targets in train_loader:
    data = data.to('cuda')
    targets = targets.to('cuda')
    preds = mymodel(data)

    loss = criterion(preds,targets)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    total_epochs_loss = total_epochs_loss + loss.item()

  avg_loss = total_epochs_loss/len(train_loader)
  print(f'Epoch: {epoch+1}, Loss: {avg_loss}')


Epoch: 1, Loss: 0.6361622224908055
Epoch: 2, Loss: 0.37079794936851146
Epoch: 3, Loss: 0.3182781113797498
Epoch: 4, Loss: 0.28133734386591686
Epoch: 5, Loss: 0.2600622508441864
Epoch: 6, Loss: 0.2371094640153748
Epoch: 7, Loss: 0.22424670924149936
Epoch: 8, Loss: 0.20648566899753645
Epoch: 9, Loss: 0.19418662950904894
Epoch: 10, Loss: 0.18210951369533787
Epoch: 11, Loss: 0.1701043485832621
Epoch: 12, Loss: 0.1586348440629836
Epoch: 13, Loss: 0.15131249825464194
Epoch: 14, Loss: 0.14313211571292817
Epoch: 15, Loss: 0.13504646164275755
Epoch: 16, Loss: 0.12524162867939193
Epoch: 17, Loss: 0.11852327609095821
Epoch: 18, Loss: 0.10940437067505938
Epoch: 19, Loss: 0.1048279445450925
Epoch: 20, Loss: 0.09768524616266658
Epoch: 21, Loss: 0.09200783389489423
Epoch: 22, Loss: 0.085710769898325
Epoch: 23, Loss: 0.08075800787069383
Epoch: 24, Loss: 0.08045887076913154
Epoch: 25, Loss: 0.07532519458863599
Epoch: 26, Loss: 0.0741288976739432
Epoch: 27, Loss: 0.0665634850818828
Epoch: 28, Loss: 0.06

In [32]:
with torch.no_grad():
   correct = 0
   total = 0
   for data, targets in test_loader:
    data, targets = data.to('cuda'), targets.to('cuda') # Move data to device
    preds = mymodel(data)
    _, predicted = torch.max(preds.data, 1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()
    accuracy = 100*correct/total
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 81.74


In [33]:
with torch.no_grad():
    correct = 0
    total = 0
    for data, targets in train_loader:
      data, targets = data.to('cuda'), targets.to('cuda') # Move data to device
      preds = mymodel(data)

      _, predicted = torch.max(preds.data, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()
      accuracy = 100*correct/total
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 99.72


# **Let's Make CNN model using pretrained model**

In [34]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/Fashion_MNIST/fashion-mnist_train.csv')
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25, random_state=69)





In [62]:
from torchvision.transforms import transforms
custom_transforms = transforms.Compose([
    transforms.Resize(232),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [65]:
from PIL import Image
import numpy as np


class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, features, labels, transform):
    self.features = features
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    image = self.features[index].reshape(28,28)
    image = image.astype(np.uint8)

    image = np.stack((image,) * 3, axis=-1)

    image = Image.fromarray(image)
    image = self.transform(image)

    return image, torch.tensor(self.labels[index], dtype = torch.long)


In [66]:
from torch.utils.data import Dataset, DataLoader

train_dataset = CustomDataset(X_train, y_train, transform = custom_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory= True)
test_dataset = CustomDataset(X_test, y_test, transform = custom_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory= True)

In [67]:
# import Pretrained Models
import torchvision.models as models
pre_model = models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [80]:
for param in pre_model.parameters():
  param.requires_grad = False

pre_model.fc = nn.Sequential(
    nn.Linear(2048,128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(64,10)
)

In [76]:
pre_model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [82]:
pre_model = pre_model.to('cuda')
learning_rate_2 = 0.0001
epochs_2 = 10

In [72]:
criterion_2 = nn.CrossEntropyLoss()
optimizer_2 = torch.optim.Adam(pre_model.fc.parameters(), lr=learning_rate_2)

In [ ]:
for epoch in range(epochs):
  total_epochs_loss = 0
  for data, targets in train_loader:
    data = data.to('cuda')
    targets = targets.to('cuda')
    preds = pre_model(data)

    loss = criterion(preds,targets)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    total_epochs_loss = total_epochs_loss + loss.item()

  avg_loss = total_epochs_loss/len(train_loader)
  print(f'Epoch: {epoch+1}, Loss: {avg_loss}')
